# Seq2Seq模型效果测试

---

In [1]:
import os
import re
from tqdm import tqdm
import traceback
import sys
import random
import pprint
import jieba
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

sys.path.insert(0, "/home/team55/notespace/zengbin")

from jddc.config import Seq2SeqConfig
import jddc.utils as u
import jddc.datasets as d
from jddc.obj import Session, Sentence

conf = Seq2SeqConfig()
logger = u.create_logger(name='seq2seq', log_file=conf.log_file, cmd=conf.cmd_log)

In [2]:
import torch
from torch.optim.lr_scheduler import StepLR
import torchtext

from jddc.seq2seq.fields import *
from jddc.seq2seq.optim import Optimizer
from jddc.seq2seq.models import EncoderRNN, DecoderRNN, Seq2seq, TopKDecoder
from jddc.seq2seq.loss import NLLLoss
from jddc.seq2seq.supervised_trainer import SupervisedTrainer
from jddc.seq2seq.checkpoint import Checkpoint
from jddc.seq2seq.predictor import Predictor
import jddc.seq2seq.main as s

## 1 - 构建数据集

---

step 1. 抽样10万个session，用create_dataset05提取QQ+A数据样本；

step 2. 抽样20万个session，用QAQAQ + A构造 QQ+A数据集


方案二：

1）6万个session全部QQA；2）6万个session的单轮QQA；3）6万个session去除首尾之后的全部QQA

In [ ]:
# 分词，保存成tsv
train_data = []
train_data_reverse_q = []
for i in tqdm(range(len(questions))): 
    q_tokens = u.jieba_tokenize(questions[i])
    a_tokens = u.jieba_tokenize(answers[i])
    row = " ".join(q_tokens) + "\t" + " ".join(a_tokens) 
    train_data.append(row)
    # 翻转src
    row_reverse_q = " ".join(q_tokens[::-1]) + "\t" + " ".join(a_tokens) 
    train_data_reverse_q.append(row_reverse_q)
    
u.write_file(conf.file_train, content=train_data, mode='w', encoding="utf-8")
u.write_file(conf.file_train_rq, content=train_data, mode='w', encoding="utf-8")

In [ ]:
# 加载数据集
fresh_data = True
if fresh_data:
    all_sessions = u.read_from_pkl(conf.pkl_sessions)
    questions1, answers1 = d.create_dataset06(all_sessions[:300000], 60000)
    questions2, answers2 = d.create_dataset05(all_sessions[300000:600000], 60000, random_sample=False)
    questions3, answers3 = d.create_dataset07(all_sessions[600000:], 60000)
    questions = questions1 + questions2 + questions3
    answers = answers1 + answers2 + answers3
    questions = [x.replace('\0','') for x in questions]
    answers = [x.replace('\0','') for x in answers]
    print(len(questions))
    u.save_to_pkl(conf.file_qa_pairs, data=[questions, answers])
else:
    questions, answers = u.read_from_pkl(conf.file_qa_pairs)

# 分词，保存成tsv
train_data = []
train_data_reverse_q = []
for i in tqdm(range(len(questions))): 
    q_tokens = u.jieba_tokenize(questions[i])
    a_tokens = u.jieba_tokenize(answers[i])
    row = " ".join(q_tokens) + "\t" + " ".join(a_tokens) 
    train_data.append(row)
    # 翻转src
    row_reverse_q = " ".join(q_tokens[::-1]) + "\t" + " ".join(a_tokens) 
    train_data_reverse_q.append(row_reverse_q)
    
u.write_file(conf.file_train, content=train_data, mode='w', encoding="utf-8")
u.write_file(conf.file_train_rq, content=train_data, mode='w', encoding="utf-8")

In [ ]:
# solution for  _csv.Error: field larger than field limit (131072)
import csv
csv.field_size_limit(500 * 1024 * 1024)

src = SourceField(batch_first=True)
tgt = TargetField(batch_first=True)
max_len = 200

def len_filter(example):
    return len(example.src) <= max_len and len(example.tgt) <= max_len

train = torchtext.data.TabularDataset(
    path=conf.file_train_rq, format='tsv',
    fields=[('src', src), ('tgt', tgt)],
    filter_pred=len_filter
)

src.build_vocab(train, max_size=200000)
tgt.build_vocab(train, max_size=100000)
input_vocab = src.vocab
output_vocab = tgt.vocab

In [ ]:
len(input_vocab)

## 2 - 定义模型
---

In [ ]:
print(conf.encoder_params)
print(conf.decoder_params)

In [ ]:
loss = NLLLoss()
encoder = EncoderRNN(vocab_size=len(src.vocab), **conf.encoder_params)
decoder = DecoderRNN(vocab_size=len(tgt.vocab), eos_id=tgt.eos_id, sos_id=tgt.sos_id, **conf.decoder_params)
seq2seq = Seq2seq(encoder, decoder)

if conf.use_cuda:
    seq2seq.cuda()
    loss.cuda()

for param in seq2seq.parameters():
    param.data.uniform_(-0.08, 0.08)

## 训练模型

---

技巧：

1. 用尽可能多的语料去训练，使用epoch设置不要超过3
2. 将question分词后翻转


In [ ]:
# Optimizer and learning rate scheduler can be customized by
# # explicitly constructing the objects and pass to the trainer.

optimizer = Optimizer(torch.optim.Adam(seq2seq.parameters()), max_grad_norm=5)
# scheduler = StepLR(optimizer.optimizer, 1)
# optimizer.set_scheduler(scheduler)

# train
trainer = SupervisedTrainer(loss=loss, batch_size=32, checkpoint_every=500, print_every=10, 
                      expt_dir=conf.s2s_path, random_seed="1234", use_cuda=conf.use_cuda)

trainer.logger = logger
seq2seq = trainer.train(seq2seq, train, num_epochs=3,
                  optimizer=optimizer,
                  teacher_forcing_ratio=0.5,
                  resume=True)

## 预测

---

In [ ]:
# questions, answers = u.read_from_pkl(conf.pkl_mqa_10000)
test_q = d.read_test_questions02(conf.file_test_q)

In [ ]:
# 加载模型
latest_checkpoint_path = Checkpoint.get_latest_checkpoint(conf.s2s_path)
print(latest_checkpoint_path)
resume_checkpoint = Checkpoint.load(latest_checkpoint_path)
model = resume_checkpoint.model
src_vocab = resume_checkpoint.input_vocab
tgt_vocab = resume_checkpoint.output_vocab

In [ ]:
model = model.cpu()
model.decoder.use_cuda = False

In [ ]:
# GPU模式下运行预测
predictor = Predictor(model, src_vocab, tgt_vocab, use_cuda=False)

In [ ]:
i = 10
print(i, "="*88+"\n", "question:", test_q[i])
print("-"*88)
results = predictor.predict_n(u.jieba_tokenize(test_q[i]), 10)
print("answer:\n")
for idx, x in enumerate(results, 1):
    print(idx, "".join(x))

In [ ]:
for i in range(0, 111):
    print(i, "="*88+"\n", "question:", test_q[i])
    print("-"*88)
    results = predictor.predict_n(u.jieba_tokenize(test_q[i]), 6)
    print("answer:\n")
    for x in results:
        print("".join(x))

## 测试模型性能

---



### seq2seq - hidden_size:128, layers: 4

2018_09_11_20_27_26 = 58500step

2018_09_12_11_18_58 = 5epoch

2018_09_12_04_21_01 = 4epoch

### seq2seq -hidden_size:256, layers:5, dropout:0.3

2018_09_13_13_05_03 = 2epoch

In [3]:
s.latest_checkpoint_path = os.path.join(conf.s2s_path, "checkpoints/2018_09_13_17_48_17")
# s.latest_checkpoint_path = os.path.join(conf.s2s_path, "small/seq2seq_small_04")

In [4]:
output_file = "/home/team55/notespace/zengbin/answers/seq2seq_answers004.txt"
s.run_prediction(conf.file_test_q, output_file, rq=True)

load model from /home/team55/notespace/data/seq2seq/checkpoints/2018_09_13_17_48_17


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.985 seconds.


1 ====================================================================
 question: 其他问题<q>我直接解绑了实名认证的那个账号，需要把手机号也解绑了 是的 金融客服说这个要找你们解决 他们不管


Prefix dict has been built succesfully.
/home/team55/notespace/zengbin/jddc/seq2seq/predictor.py:86: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  tgt_id_seq = [other['topk_sequence'][di][0, x, 0].data[0] for di in range(length)]


answer:

2 亲爱的，发送验证码给您了，[数字x]
3 #E-s[数字x]
4 请问还有其他可以帮到您的吗?#E-s[数字x]
5 亲爱滴，请问还有其他需要帮助的么?#E-b[数字x]
6 亲爱滴，请问还有其他需要帮助的么?#E-s[数字]
2 ====================================================================
 question: ok<q>新电话号码+[电话x]
answer:

2 #E-s[数字您]
3 #E-s[数字的]
4 很高兴遇到您这么善解人意的客户，请问还有其他还可以帮到您的吗#E-s[数字x]
5 很高兴遇到您这么善解人意的客户，请问还有其他可以帮到您的吗?#E-s[数字x]
6 请问还有其他还可以帮到您的吗?#E-s[数字x]
3 ====================================================================
 question: [数字x] 这个订单的发票什么时候开呢<q>[数字x] 这两个订单号
answer:

2 您好，关于您反馈的问题，为了更好的帮您解答，我需要帮您反馈配送部门核实情况，核实的结果专员会在[数字x]
3 您好，关于您反馈的问题，为了更好的帮您解答，我需要帮您反馈相关部门核实情况，专员会在[数字x]
4 您好，关于您反馈的问题，为了更好的帮您解答，我需要帮您反馈相关部门核实情况，预计[数字x]
5 您好，关于您反馈的问题，为了更好的帮您解答，我需要帮您反馈相关部门核实情况，核实后会在[数字x]
6 您好，关于您反馈的问题，为了更好的帮您解答，我需要帮您反馈配送部门核实情况，核实后会在[数字x]
4 ====================================================================
 question: 我们家下午六点取件，师傅说[数字x]点他们取不了<q>我想把这个退了，重新买个
answer:

2 这边帮您备注一下
3 这边帮您备注一下哦
4 这边帮您联系站点核实哦
5 这边帮您联系站点核实一下呢
6 这边帮您联系站点核实一下哦
5 =================================================

answer:

2 [姓名x]
3 亲爱的，这边帮您升级专员处理，[数字x]
4 亲爱的，这边帮您升级专员处理哦
5 亲爱的，这边帮您升级专员处理，专员会在[数字x]
6 亲爱的，这边帮您升级专员处理，专员[数字x]
29 ====================================================================
 question: 这个地址能到吗 亲爱的<q>亲爱的 好的
answer:

2 好的呢亲
3 好的呢亲
4 好的呢亲
5 好的呢亲
6 好的呢亲
30 ====================================================================
 question: 是<q>好的，谢谢您! 对，只有一箱
answer:

2 这边帮您申请售后可以吗
3 这边帮您申请售后您看可以吗
4 这边帮您申请售后您看可以吗?
5 这边帮您申请售后可以吗?
6 这边帮您申请售后您看可以吗?#E-s[数字x]
31 ====================================================================
 question: [姓名x]，天津[地址x] [数字x]<q>买过斑布的卫生纸 还有书
answer:

2 [数字x]
3 [姓名x]
4 #E-s[数字]
5 #E-s[数字您]
6 #E-s[数字的]
32 ====================================================================
 question: 你这个手表是插什么卡的? 移动的吗?<q>手表软件有游戏吗?
answer:

2 亲，这个是没有内存卡的呢
3 亲，这个是没有内存卡的哦
4 亲，这个是没有内存卡的呢
5 亲，这个是没有内存卡的呢
6 亲，这个是没有的呢
33 ====================================================================
 question: E卡都是能正常开发票的对吗 之前的问题<q>我的意思是，公司打算采购一些京东E卡，这个是能正常开发票的吗 增票还是?
answer:

2 亲爱的，增票开出的周期是您资质审核

answer:

2 #E-s[数字祝您]
3 #E-s[数字您]
4 感谢您对京东的支持，祝您生活#E-s[数字x]
5 感谢您对京东的支持，祝您生活愉快，再见#E-s[数字x]
6 #E-s[数字x生活#E-s[数字x]
56 ====================================================================
 question: 查了没<q>好
answer:

2 还辛苦您给小妹打个评价呢~点击表情栏旁边的“+”哦。#E-s[数字x]
3 妹子祝福您幸福快乐，前程锦绣，还请您点击表情栏旁边的“+”打赏我一个评价哦#E-s[数字x]
4 #E-s[数字祝您]
5 还辛苦您给小妹打个评价呢~#E-s[数字x]
6 #E-s[数字x生活#E-s[数字x]
57 ====================================================================
 question: 好<q>反正你们是京东客服
answer:

2 #E-s[数字x]
3 嗯嗯请问还有其他还可以帮到您的吗?#E-s[数字x]
4 #E-s[数字您]
5 嗯嗯请问还有其他还可以帮到您的吗?感谢您对京东的支持，祝您生活愉快，再见!#E-s[数字x]
6 嗯嗯请问还有其他还可以帮到您的吗?还辛苦您给小妹打个评价呢~点击表情栏旁边的“+”哦。#E-s[数字x]
58 ====================================================================
 question: 反正你们是京东客服<q>相信你们会处理好
answer:

2 请问还有其他还可以帮到您的吗?
3 #E-s[数字x]
4 嗯嗯好的
5 请问还有其他可以帮到您的吗?#E-s[数字x]
6 请问还有其他还可以帮到您]
59 ====================================================================
 question: 相信你们会处理好<q>至少得发货吧 不能老是说什么促销啊，爆仓啊之类的 是
answer:

2 请问还有其他还可以帮到您的吗?#E-s[数字x]
3 #E-s[数字x]
4 请问还有其他还可以帮到您]


answer:

2 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~#E-s[数字x]
3 亲爱的客户，还麻烦您提供下订单号，妹子这边给您查询哦~#E-s[数字x]
4 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~#E-j[数字x]
5 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~]
6 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~]
90 ====================================================================
 question: 为什么小米[数字x]没有发货 小姐姐<q>是呀 为什么别人都发货了
answer:

2 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~#E-s[数字x]
3 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~#E-j[数字x]
4 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~]
5 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~]
6 亲爱的客户，APP端麻烦您点击对话框右下角的“+”，点击“订单”后，选择一下您需要咨询的订单哦，PC端在我的订单复制下哦，小妹这边帮您查询一下哦~]
91 ====================================================================
 question: 是呀 为什么别人都发货了<q>�那我